https://www.1point3acres.com/bbs/thread-831026-1-1.html

https://www.1point3acres.com/bbs/thread-1032751-1-1.html

https://www.1point3acres.com/bbs/thread-780948-2-1.html

https://www.1point3acres.com/bbs/thread-771600-1-1.html

### Problem: 

our employee might have written "Y Y N Y", which means the store was open for four hours that day, and it had customers shopping during every hour but its third one.
```
  hour: | 1 | 2 | 3 | 4 |
  log:  | Y | Y | N | Y |
                  ^
                  |
            No customers during hour 3
```

We suspect that we're keeping the store open too long, so we'd like to understand when we *should have* closed the store. For simplicity's sake, we'll talk about when to close the store by talking about how many hours it was open: if our closing time is `2`, that means the store would have been open for two hours and then closed.
```
  hour:         | 1 | 2 | 3 | 4 |
  log:          | Y | Y | N | Y |
  closing_time: 0   1   2   3   4
                ^               ^
                |               |
         before hour #1    after hour #4
```

(A closing time of 0 means we simply wouldn't have opened the store at all that day.)

First, let's define a "penalty": what we want to know is "how bad would it be if we had closed the store at a given hour?" For a given log and a given closing time, we compute our penalty like this:

  +1 penalty for every hour that we're *open* with no customers
  +1 penalty for every hour that we're *closed* when customers would have shopped
```
For example:
  hour:    | 1 | 2 | 3 | 4 |   penalty = 3:
  log:     | Y | Y | N | Y |     (three hours with customers after closing)
  penalty: | * | * |   | * |
           ^
           |
         closing_time = 0
  hour:    | 1 | 2 | 3 | 4 |   penalty = 2:
  log:     | N | Y | N | Y |      (one hour without customers while open +
  penalty: | * |   |   | * |       one hour with customers after closing)
                   ^
                   |
                 closing_time = 2
  hour:    | 1 | 2 | 3 | 4 |   penalty = 1
  log:     | Y | Y | N | Y |      (one hour without customers while open)
  penalty: |   |   | * |   |
                           ^
                           |
                         closing_time = 4
```
Note that if we have a log from `n` open hours, the `closing_time` variable can range from 0, meaning "never even opened", to n, meaning "open the entire time".

### 1a)
Write a function `compute_penalty` that computes the total penalty, given
  
a store log (as a space separated string) AND

a closing time (as an integer)

In addition to writing this function, you should use tests to demonstrate that it's correct. Do some simple testing, and then quickly describe a few other tests you would write given more time.

#### Examples: 
```python
compute_penalty("Y Y N Y", 0) should return 3
compute_penalty("N Y N Y", 2) should return 2
compute_penalty("Y Y N Y", 4) should return 1
```


In [22]:
def compute_penalty_part1(log, closing_time):

    log_list = log.split(' ')
    # print(log_list)

    penalty = 0
    for i in range(len(log_list)):
        # print(log_list[i])
        # print(i)
        if i >= closing_time and log_list[i] == 'Y':
            # print("Y after closing time")
            penalty += 1
        elif i < closing_time and log_list[i] == 'N':
            # print("N before closing time")
            penalty += 1

    return penalty



In [23]:
print(compute_penalty_part1("Y Y N Y", 0)) # should return 3
print(compute_penalty_part1("N Y N Y", 2)) # should return 2
print(compute_penalty_part1("Y Y N Y", 4)) # should return 1

3
2
1


### 1b)
Write another function named `find_best_closing_time` that returns the best closing time in terms of `compute_penalty` given just a store log. You should use your answer from 1a to solve this problem.

Again, you should use tests to demonstrate that it's correct. Do some simple testing, and then quickly describe a few other tests you would write given more time.  

In [68]:
def find_best_closing_time(log):

    log_list = log.split(' ')
    N = len(log_list)+1

    min_penalty = float('inf')
    best_time = 0
    
    for i in range(N):
        curr_penalty = compute_penalty_part1(log, i)
        if curr_penalty < min_penalty:
            best_time = i
            min_penalty = min(curr_penalty, min_penalty)
    
    return best_time
        


In [69]:
find_best_closing_time("Y Y N N") # should return 2

2

### 2a)
We've asked our employees to write their store logs all together in the same text file, marking the beginning of each day with `BEGIN` and the end of each day as `END`, sometimes spanning multiple lines. We hoped that the file might look like
```
"BEGIN Y Y END \nBEGIN N N END"
```
which would represent two days, where the store was open two hours each day. Unfortunately, our employees are often too busy to remember to finish the logs, which means this text file is littered with unfinished days and extra information scattered throughout. Luckily, we know that an unbroken sequence of BEGIN, zero or more Y's or N's, and END is going to be a valid log, so we can search the aggregate log for those.

For example, given the aggregate log
  
```
"BEGIN BEGIN BEGIN N N BEGIN Y Y Y END N N END"
                         ^           ^
                         |           |
                         +-----------+
                           valid log
```

We can extract only one valid sequence, "BEGIN Y Y Y END". For our purposes, we should ignore any invalid sequences. *These logs cannot be nested.*

Write a function `get_best_closing_times` that takes an aggregate log as a string and returns an array of best closing times for every valid log we can find, in the order that we find them.
Again, you should use tests to demonstrate that it's correct. Do some simple testing, and then quickly describe a few other tests you would write given more time.

```
## Examples
get_best_closing_times("BEGIN Y Y END \nBEGIN N N END")
  should return an array: [2, 0]
get_best_closing_times("BEGIN BEGIN \nBEGIN N N BEGIN Y Y\n END N N END")
  should return an array: [2]
```

In [72]:
def get_best_closing_times(agg_log):

    # parse the aggregated log to get valid log
    agg_log_list = agg_log.split()
    # print(agg_log_list)

    log_list = []
    curr_log = []
    valid_log_flag = False
    for i in agg_log_list:
        # potential start of a valid log, clear the curr_log and set the flag to True
        if i == 'BEGIN':
            valid_log_flag = True
            curr_log = []
        elif i in ['Y', 'N'] and valid_log_flag:
            curr_log.append(i)
        elif i == 'END' and valid_log_flag:
            log_list.append(curr_log)
            valid_log_flag = False
    # print(log_list)

    result = []
    for i in range(len(log_list)):
        # call the function from last question
        result.append(find_best_closing_time(" ".join(log_list[i])))

    return result

    

In [73]:
print(get_best_closing_times("BEGIN Y Y END \nBEGIN N N END")) # should return an array: [2, 0]
print(get_best_closing_times("BEGIN BEGIN \nBEGIN N N BEGIN Y Y\n END N N END")) # should return an array: [2]

[2, 0]
[2]


In [ ]:
# usage of split function, if the delimiter is not specified - By default any whitespace is a separator, including \n
logs = "BEGIN Y Y END \nBEGIN N N END"
logs.split()

['BEGIN', 'Y', 'Y', 'END', 'BEGIN', 'N', 'N', 'END']

### 2b)
Over time these aggregate log files can get very large, so write another function, get_best_closing_times__v2. This should work like the previous one, but with two important differences:

Instead of accepting an aggregate log as a string, it should accept a filename (and not the contents of the file.)

It should take care not to read the whole file’s contents at once, and instead should stream the file as it goes. Assume that the whole file is too large to fit in memory. (You can, however, safely assume that each individual line of the file can fit in memory.)

In all other respects, this function should behave like get_best_closing_times, finding the valid log sequences and returning an array of the best closing times corresponding to each.

```python
Example
# assuming `myfile.txt` contains "BEGIN BEGIN \nBEGIN N N BEGIN Y Y\n END N N END"
get_best_closing_times__v2("myfile.txt") # should return an array: [2]
```

In [92]:
def get_best_closing_times__v2(file_path):

    result = []
    valid_log = []
    valid_log_flag = False
    with open(file_path, "r") as file:
        for line in file:
            log = line.replace('\\n', '').split()

            for i in log:
                if i == 'BEGIN':
                    valid_log_flag = True
                    valid_log = []
                elif i in ['Y', 'N'] and valid_log_flag:
                    valid_log.append(i)
                elif i == 'END' and valid_log_flag:
                    # call the previous function to find best closing time
                    best_time = find_best_closing_time(" ".join(valid_log))
                    result.append(best_time)
                    valid_log_flag = False

    return result


In [93]:
get_best_closing_times__v2("logs/store_logs_v1.txt")

[2, 2, 0]